In [1]:
from fh import FH 
import fh
from treeBuilder import TreeBuilder as TB
import PydalChanel as pydal
import copy
import pickle
import arpeggiator
from spatializer import Spatializer

lp = FH()


oneHitShift = fh.oneHitShift

read = pydal.read
end = pydal.end
tempo = pydal.tempo
rootScale = lp.rootScale
stripSymbols = lambda beats : map(lambda beat : list(beat[1])[0], beats)

ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)
ch3 = pydal.newChannel(3)
ch4 = pydal.newChannel(4)
ch5 = pydal.newChannel(5)

idv = lambda a : a
codec = lambda buf: fh.noteListToHitList(fh.flattenByBeat(fh.notesByBeat(fh.hitListToNoteList(buf))))

In [2]:
s = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp)
s.spatialize = False
s.separateChannels = True
lp.addForwardingHandler(1, s)
s.loadChordsFromFile("droneSong3.chords")
lp.loadScenesFromFile("droneSongParams.scenes")
lp.playScene(1)
def toggleSustain():
    s.sustaining = not s.sustaining
def toggleSeparateChannels():
    s.separateChannels = not s.separateChannels
lp.topRowFunctions[0] = toggleSustain
lp.topRowFunctions[1] = toggleSeparateChannels
# have 3 rows of chords
# 1 - theme1, 2 - variation, 3 - "climax" of theme1 statement
# need - theme1 variant with strongly resolving cadence
#      - theme2 - a minor theme (borrow/rearrange a minor sequence from theme 1, add diminished chords?)
#      - variation on theme2
#      - a similar "climax" on theme 2 - strong movements of a minor flavor
#      - a transition to or restatement of the "climax" of theme 1 
#      - a restatement of the theme1 variant with the most strongly resolving cadence (last 3 chords already written)
# TODO - add option to leave shared notes between chord transitions on to allow longer synth decay
#      - find (at least) 4 different parameter points for the synth 
#      - think of some temporal scheme/pattern for how the synth sound changes

# note - move keys "down" a row to access lower notes and visualize chords properly

# 8 "lines"
# l,2,3 - rows 1,2,3 - 1 beat per chord, 2 beats on last
# 4 - row 3 - 1248537 - 1 beat per chord, 2 beats on last
# 5,6 - row 4 - 12545456 then 1-8
# 7 - row 1 - 123, row 3 - 34567 1 beat per chord, 2 beats on last
# 8 - row 3 - 1234, row 6 - 567 1 beat per chord, 2 beats on last

In [ ]:
s.playChord(s.savedChords[5])

In [ ]:
lp.saveScenesToFile("droneSongParams.scenes")

In [ ]:
'''
bd 3
lt
sn 5
ho
hc 
bot 1
bin 
bend
2 freebie - rd:1? bin:1? cp? bend? sn3?
'''

a = read('[bd sn]*2 rd:1 [bd sn]*2 [rd:1 ~ ~ rd:1] [~ rd:1]', 5)
a1 = read('[~ <sn, ~>]*2 bin [~ sn]*2 [rd:1 ~ ~ bin] [~ rd:1]', 5)
b = read("~ [lt lt ~]", 1.5)
c = read("bd:3 <bd:3, bd> ~")
br = read("(bd*2, [bd:3 bd:3] bd) [sn:5 [bd sn:3]] [[bd sn:3] bd] <sn:5 [bd sn:3], <sn:5*4, sn:5*2 ~, sn:5*2 [sn:5 ~]>>", 4)
d = read("bd:3 bd:3 <sn:5, ~, bd:3*2> <bd:3, bd:3*2> bd:3 sn:5 bd sn:5*2", 4)
e = read("{[bin <bin:1, bin, bin*2> bin bot, hc <hc, ho>], bd sn:3}")
f = read("[hc ~] [bend ~] ([~ bin], [bin ~])", 3)
g = read("((lt,  bd, <lt, bd>, <lt, bd>)*4, sn:3 (~, <sn:3, ~>))")
h = read("{lt*3, <hc hc, bd sn>, (cp cp:3, bin) [bd sn]}")
demo = read("{lt bd bd*3, hc ho}")

In [ ]:
ch1.stop()

In [ ]:
def sendParams(bank, visualize=True, sonify=False):
    banksToString = lambda a: "-".join(map(lambda bank: ".".join(map(lambda slot: ",".join(map(str,slot)), bank)), a))
    currentFadersToString = lambda bank: ".".join(map(lambda slot: ",".join(map(str,slot)), bank))
    if visualize:
        lp.currentFaderVals[2] = bank
    if sonify:
        lp.currentFaderVals[1] = bank
    msg = OSC.OSCMessage()
    msg.setAddress("/loadSceneFaders")
    msg.append(banksToString(lp.faderBanks))
    msg.append(currentFadersToString(lp.currentFaderVals))
    lp.superColliderClient.send(msg)

In [ ]:
# coolParams = []
coolParams.append(lp.currentFaderVals[1])

In [ ]:
sendParams([0]*12, sonify=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 12
menMeans = [60]*12

ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, menMeans, width, color='r')


# add some text for labels, title and axes ticks
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.set_xticks(ind + width)
ax.set_xticklabels(('G1', 'G2', 'G3', 'G4', 'G5'))

ax.legend((rects1[0]), ('params'))


def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)

plt.show()

In [ ]:
s.heldNotesOff(1)

In [ ]:
lp.startChannel(1, lambda buf: fh.spatialize(buf, lp.roots[0], [[0,1,2,3], [4,5,6,7], [8,9,10,11]]), lp.loops[0][0])

In [ ]:
lp.stopChannel(1)

In [ ]:
a = read("{bd (lt*2, lt, lt), hc <~, sn hc> ~ hc:2}")
b = read("{(bd ~, sn:3 <~, cp>)}")
c = read("((bd:2*2, [bd:3 bd:3] bd), [sn:5 [bd sn:3]], [[bd sn:3] bd], <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>)")
c2 = read("(bd:2*2, [bd:3 bd:3] bd) [sn:5 [bd sn:3]] [[bd sn:3] bd] <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>", 4)
d = read("{~ ~, bin ~ bin ~}")
e = read("{[bin <bin:1, bin, bin*2> bin bot, hc <hc, ho>], bd sn:3}")

'''
bd 3
lt
sn 5
cp
hc:1 
ho
bot
bin 
bend:2
1 freebie 
'''

demo = read("bend:2 bin")

f = read("aa:4_4.75*2 aa:5_5.75*3", 2, 'sample')
f2 = read("hc*2 cp*3", 2)
d2 = read("{~ ~ , bd ~ sn:3 ~}", 4)
g = read("{~ ~ , ab:0_2 ~ ab:3_5 ~}", 4, 'sample')

In [ ]:
ch1.stop()

In [ ]:
s.saveChordsToFile("droneSong3.chords")

In [ ]:
rootScale(1, lp.roots[1], "maj5")

In [ ]:
lp.startChannel(1, lambda buf: fh.randTranspose(buf, lp.roots[1], lp.scales[1]), lp.loops[1][0])

In [ ]:
lp.stopChannel(1)

In [ ]:
lp.startChannel(1, lambda buf: fh.warp(buf, 3, 1), lp.loops[1][3])